In [2]:
import pandas as pd
import altair as alt

# Load the data
df = pd.read_csv("cleaned_rental_data.csv")

# Rename columns for clarity
df = df.rename(columns={
    "Total Rent Avg": "Rent",
    "Utility Cost Avg": "Utilities"
})

# Melt to long format
melted_df = df.melt(
    id_vars=["Neighborhood"],
    value_vars=["Rent", "Utilities"],
    var_name="Cost Type",
    value_name="Cost"
)

# Group by Neighborhood and Cost Type and calculate averages
avg_costs = melted_df.groupby(["Neighborhood", "Cost Type"], as_index=False)["Cost"].mean()

# Sort neighborhoods by total cost
sorted_neighborhoods = (
    avg_costs.pivot(index="Neighborhood", columns="Cost Type", values="Cost")
    .fillna(0)
    .assign(Total=lambda x: x["Rent"] + x["Utilities"])
    .sort_values("Total")
    .index.tolist()
)

# Define green color scale
color_scale = alt.Scale(domain=["Rent", "Utilities"], range=["#A8D5BA", "#5EAD90"])

# Create the stacked bar chart
chart = alt.Chart(avg_costs).mark_bar().encode(
    x=alt.X("Neighborhood:N", sort=sorted_neighborhoods, title="Neighborhood"),
    y=alt.Y("Cost:Q", stack="zero", title="Average Monthly Housing Cost"),
    color=alt.Color("Cost Type:N", title="Cost Type", scale=color_scale),
    tooltip=["Neighborhood", "Cost Type", "Cost"]
).properties(
    title="Average Rent and Utility Costs by Neighborhood",
    width=750,
    background="#FFFDE7"  # Light cream background
).configure_axisX(
    labelAngle=-40
).configure_title(
    fontSize=18,
    anchor='start',
    color='black'  # Black title
).configure_view(
    stroke=None
)

chart


alt.Chart(...)

In [3]:
# HTML
chart.save("rent_utilities_by_neighborhood.html")
